In [1]:
"""Help file to calculate errors and adjust the CSV data."""

import os

# scipy libs
import numpy as np
import pandas as pd
from scipy import interpolate

import pdb

In [2]:
def V_error(V, vertical_position=0, div=1):
    """Calculates the error for voltages."""
    err_percent = 0.3 / 100
    V_adj = V + vertical_position
    div_err = div / 1000
    fix_err = 7 / 1000
    
    calibration = err_percent * V_adj + div_err + fix_err
    resolution = div

    return np.sqrt(calibration**2 + resolution**2)


def t_error(t, div=None):
    """Error of time."""
    samples = len(t)

    if not div:
        div = (t[samples-1] - t[0]) / (samples - 1)

    total_t = samples * div
    err_frac = 50 / 1000000  # 50 ppm
    
    return np.zeros(samples) + total_t * err_frac



In [3]:
def res_adj(value, resolution, rand=False):
    value = resolution * np.round(value/resolution)
    
    if rand:
        deviation = np.random.choice([0, 1, 2, 3], size=len(value), p=[.45, .3, .2, .05])
        direction = np.random.choice([-1, 1], size=len(value))        
        value = value + resolution * deviation * direction

    return value


def spline(data, points=200, reduce=True):
    if reduce:
        keys = data.keys()
        new_data = dict()
        for key in keys:
            first_part = np.array(data[key][::2])
            second_part = np.array(data[key][1::2])
            if len(first_part) > len(second_part):
                first_part = first_part[:len(second_part)]

            new_data[key] = (first_part + second_part) / 2

        data = pd.DataFrame(new_data, dtype=np.float64)

    ip_vi = interpolate.interp1d(data["t"], data["Vi"], kind="cubic")
    ip_vo = interpolate.interp1d(data["t"], data["Vo"], kind="cubic")

    new_data = pd.DataFrame(columns=("t", "Vi", "Vo"), dtype=np.float64)
    new_data["t"] = np.linspace(min(data["t"]), max(data["t"]), points)
    new_data["Vi"] = ip_vi(new_data["t"])
    new_data["Vo"] = ip_vo(new_data["t"])

    return new_data


In [4]:
path = "dados/sim/"
save_path = "dados/"

dec = '.'
name_changes = {
    "time": "t",
    "vin": "Vi",
    "vout": "Vo"
}


In [5]:
res_in = 10
res = {
    "quad_deriv": 5,
    "quad_integ": 1,
    "sin_deriv": 1,
    "sin_integ": 1,
    "tri_deriv": 1,
    "tri_integ": 1,
}

for file in os.listdir(path):
    if "crit" in file:
        continue
    elif "o.csv" in file:
        res_out = res_in
    else:
        res_out = res[file.replace(".csv", "")]

    # get data and adjust it
    data = pd.read_csv(path + file, decimal=dec, float_precision="high", dtype=np.float64)
    data = data.rename(index=str, columns=name_changes)

    # units and resolution adjustment
    data["t"] = data["t"] * 1000
    data["Vi"] = res_adj(data["Vi"] * 1000, res_in, True)
    data["Vo"] = res_adj(data["Vo"] * 1000, res_out, True)

    # interpolation approximation
    data = spline(data, points=100)

    # error calculations
    data["Vir"] = V_error(data["Vi"], div=5*res_in)
    data["Vor"] = V_error(data["Vo"], div=5*res_out)
    data["tr"] = t_error(data["t"])
    
    cols = ["t", "tr", "Vi", "Vir", "Vo", "Vor"]
    data = data[cols]

    # save new table for plotting
    data.to_csv(save_path + file, index=False)

    print("Done:", file)

Done: tri_integ.csv
Done: quad_integ.csv
Done: 1021o.csv
Done: quad_deriv.csv
Done: 41o.csv
Done: tri_deriv.csv
Done: sin_deriv.csv
Done: sin_integ.csv
